

Install dependencies and download data. Extract zipped data



In [1]:
!pip3 install tensorlayer==1.7.1
!pip install scipy==1.1.0
!mkdir data_faces
#!mkdir data_faces && wget https://s3-us-west-1.amazonaws.com/udacity-dlnfd/datasets/celeba.zip
#with zipfile.ZipFile("celeba.zip","r") as zip_ref:
#  zip_ref.extractall("data_faces/")
from google.colab import drive
drive.mount('/content/drive')
import zipfile
with zipfile.ZipFile("/content/drive/My Drive/VaeOnCelebA/celeba.zip","r") as zip_ref:
  zip_ref.extractall("data_faces/")

     |████████████████████████████████| 266kB 6.8MB/s 
  Created wheel for tensorlayer: filename=tensorlayer-1.7.1-cp36-none-any.whl size=256475 sha256=448774ecda29b94e4159dcf9aae58a414481826b7cacd6b77ddc63976c102747
  Stored in directory: /root/.cache/pip/wheels/f9/c0/50/7e910fcbcd85d3de5c0504bd3d4d3fc5425c72b096bd310b45
Successfully built tensorlayer
     |████████████████████████████████| 31.2MB 101kB/s 
ERROR: umap-learn 0.4.4 has requirement scipy>=1.3.1, but you'll have scipy 1.1.0 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement scipy==1.4.1; python_version >= "3", but you'll have scipy 1.1.0 which is incompatible.
ERROR: plotnine 0.6.0 has requirement scipy>=1.2.0, but you'll have scipy 1.1.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
Drive already moun

Import necessary packages, mount google drive and import necessary auxiliary python files

In [2]:
import os
import scipy.misc
import pprint
import numpy as np
import time
import sys
sys.path.append('/content/drive/My Drive/VaeOnCelebA/')
import math
%tensorflow_version 1.x
import tensorflow as tf
import tensorlayer as tl
from tensorlayer.layers import *
from glob import glob
from random import shuffle
from google.colab import files
from utils import *
from vgg_loss import *

TensorFlow 1.x selected.




Check that GPU is found. Make sure to enable GPU under "edit"

In [3]:
device_name = tf.test.gpu_device_name()
print(device_name)

/device:GPU:0


Setup model parameters

In [ ]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [ ]:
flags = tf.app.flags
flags.DEFINE_integer("epoch", 50, "Epoch to train [5]") 
flags.DEFINE_float("learning_rate", 0.0005, "Learning rate of for adam [0.001]")
flags.DEFINE_float("beta1", 0.5, "Momentum term of adam [0.5]")
flags.DEFINE_integer("train_size", 40000000000000, "The size of train images [np.inf]")
flags.DEFINE_integer("batch_size", 64, "The number of batch images [64]")
flags.DEFINE_integer("image_size", 148, "The size of image to use (will be center cropped) [108]")
flags.DEFINE_integer("output_size", 64, "The size of the output images to produce [64]")
flags.DEFINE_integer("sample_size", 64, "The number of sample images [64]")
flags.DEFINE_integer("c_dim", 3, "Dimension of image color. [3]")
flags.DEFINE_integer("z_dim", 100, "Dimension of latent representation vector from. [2048]")
flags.DEFINE_integer("sample_step", 900, "The interval of generating sample. [300]")
flags.DEFINE_integer("save_step", 900, "The interval of saveing checkpoints. [500]")
flags.DEFINE_string("dataset", "celebA", "The name of dataset [celebA]")
flags.DEFINE_string("test_name", "dfc_vae3", "The number of experiment [test2]")
flags.DEFINE_string("checkpoint_dir", "parameters_dfc", "Directory name to save the checkpoints [checkpoint]")
flags.DEFINE_string("sample_dir", "samples_dfc", "Directory name to save the image samples [samples]")
flags.DEFINE_boolean("is_train", False, "True for training, False for testing [False]")
flags.DEFINE_boolean("is_crop", True, "True for training, False for testing [False]")
flags.DEFINE_boolean("load_pretrain",True, "Default to False;If start training on a pretrained net, choose True")
FLAGS = flags.FLAGS
samples_dir = FLAGS['sample_dir'].value + "/" + FLAGS['test_name'].value
parameters_dir = FLAGS['checkpoint_dir'].value + "/" + FLAGS['test_name'].value


In [ ]:

def encoder(input_imgs, is_train = True, reuse = False):
    '''
    input_imgs: the input images to be encoded into a vector as latent representation. size here is [b_size,64,64,3]
    '''
    z_dim = FLAGS['z_dim'].value # 100
    ef_dim = 32 # encoder filter number

    w_init = tf.random_normal_initializer(stddev=0.02)
    gamma_init = tf.random_normal_initializer(1., 0.02)

    with tf.variable_scope("encoder", reuse = reuse):
        tl.layers.set_name_reuse(reuse)

        net_in = InputLayer(input_imgs, name='en/in') # (b_size,64,64,3)
        net_h0 = Conv2d(net_in, ef_dim, (4, 4), (2, 2), act=None,
                padding='SAME', W_init=w_init, name='en/h0/conv2d')
        net_h0 = BatchNormLayer(net_h0, act=lambda x: tl.act.lrelu(x, 0.2),
                is_train=is_train, gamma_init=gamma_init, name='en/h0/batch_norm')

        net_h1 = Conv2d(net_h0, ef_dim*2, (4, 4), (2, 2), act=None,
                padding='SAME', W_init=w_init, name='en/h1/conv2d')
        net_h1 = BatchNormLayer(net_h1, act=lambda x: tl.act.lrelu(x, 0.2),
                is_train=is_train, gamma_init=gamma_init, name='en/h1/batch_norm')

        net_h2 = Conv2d(net_h1, ef_dim*4, (4, 4), (2, 2), act=None,
                padding='SAME', W_init=w_init, name='en/h2/conv2d')
        net_h2 = BatchNormLayer(net_h2, act=lambda x: tl.act.lrelu(x, 0.2),
                is_train=is_train, gamma_init=gamma_init, name='en/h2/batch_norm')

        net_h3 = Conv2d(net_h2, ef_dim*8, (4, 4), (2, 2), act=None,
                padding='SAME', W_init=w_init, name='en/h3/conv2d')
        net_h3 = BatchNormLayer(net_h3, act=lambda x: tl.act.lrelu(x, 0.2),
                is_train=is_train, gamma_init=gamma_init, name='en/h3/batch_norm')

        # mean of z
        net_h4 = FlattenLayer(net_h3, name='en/h4/flatten')
        net_out1 = DenseLayer(net_h4, n_units=z_dim, act=tf.identity,
                W_init = w_init, name='en/out1/lin_sigmoid')
        z_mean = net_out1.outputs # (b_size,100)

        # log of variance of z(covariance matrix is diagonal)
        net_h5 = FlattenLayer(net_h3, name='en/h5/flatten')
        net_out2 = DenseLayer(net_h5, n_units=z_dim, act=tf.identity,
                W_init = w_init, name='en/out2/lin_sigmoid')
        z_log_sigma_sq = net_out2.outputs + 1e-6

    return net_out1, net_out2, z_mean, z_log_sigma_sq

def generator(inputs, is_train = True, reuse = False):
    image_size = FLAGS['output_size'].value # 64 the output size of generator
    s2, s4, s8, s16 = int(image_size/2), int(image_size/4), int(image_size/8), int(image_size/16) # 32,16,8,4
    gf_dim = 32
    c_dim = FLAGS['c_dim'].value # n_color 3
    batch_size = FLAGS['batch_size'].value # 64

    w_init = tf.random_normal_initializer(stddev=0.02)
    gamma_init = tf.random_normal_initializer(1., 0.02)

    with tf.variable_scope("generator", reuse = reuse):
        tl.layers.set_name_reuse(reuse)

        net_in = InputLayer(inputs, name='g/in')
        net_h0 = DenseLayer(net_in, n_units=gf_dim*8*s16*s16, W_init=w_init,
                act = tf.identity, name='g/h0/lin')
        net_h0 = ReshapeLayer(net_h0, shape=[-1, s16, s16, gf_dim*8], name='g/h0/reshape')
        net_h0 = BatchNormLayer(net_h0, act=lambda x: tl.act.lrelu(x, 0.2), is_train=is_train,
                gamma_init=gamma_init, name='g/h0/batch_norm')

        # upsampling
        net_h1 = UpSampling2dLayer(net_h0, size=[8, 8], is_scale=False, method=1, 
                                    align_corners=False, name='g/h1/upsample2d')
        net_h1 = Conv2d(net_h1, gf_dim*4, (3, 3), (1, 1), padding='SAME', W_init=w_init, name='g/h1/conv2d')
        net_h1 = BatchNormLayer(net_h1, act=lambda x: tl.act.lrelu(x, 0.2), is_train=is_train,
                gamma_init=gamma_init, name='g/h1/batch_norm')

        net_h2 = UpSampling2dLayer(net_h1, size=[16, 16], is_scale=False, method=1, 
                                    align_corners=False, name='g/h2/upsample2d')
        net_h2 = Conv2d(net_h2, gf_dim*2, (3, 3), (1, 1), padding='SAME', W_init=w_init, name='g/h2/conv2d')
        net_h2 = BatchNormLayer(net_h2, act=lambda x: tl.act.lrelu(x, 0.2), is_train=is_train,
                gamma_init=gamma_init, name='g/h2/batch_norm')

        net_h3 = UpSampling2dLayer(net_h2, size=[32, 32], is_scale=False, method=1, 
                                    align_corners=False, name='g/h3/upsample2d')
        net_h3 = Conv2d(net_h3, gf_dim, (3, 3), (1, 1), padding='SAME', W_init=w_init, name='g/h3/conv2d')
        net_h3 = BatchNormLayer(net_h3, act=lambda x: tl.act.lrelu(x, 0.2), is_train=is_train,
                gamma_init=gamma_init, name='g/h3/batch_norm')
        net_h4 = UpSampling2dLayer(net_h3, size=[64, 64], is_scale=False, method=1, 
                                    align_corners=False, name='g/h4/upsample2d')
        net_h4 = Conv2d(net_h4, c_dim, (3, 3), (1, 1), padding='SAME', W_init=w_init, name='g/h4/conv2d')
        logits = net_h4.outputs
        net_h4.outputs = tf.nn.tanh(net_h4.outputs)
    return net_h4, logits

In [ ]:

with tf.device("/gpu:0"):
  ##========================= DEFINE MODEL ===========================##
  # The input_imgs are input for both encoder and discriminator
  input_imgs = tf.placeholder(tf.float32,[FLAGS['batch_size'].value, FLAGS['output_size'].value, 
      FLAGS['output_size'].value, FLAGS['c_dim'].value], name='real_images')

  # Normal distribution for generator
  z_p = tf.random_normal(shape=(FLAGS['batch_size'].value, FLAGS['z_dim'].value), mean=0.0, stddev=1.0)
  # Normal distribution for reparameterization trick
  eps = tf.random_normal(shape=(FLAGS['batch_size'].value, FLAGS['z_dim'].value), mean=0.0, stddev=1.0)
  lr_vae = tf.placeholder(tf.float32, shape=[])

  # ----------------------encoder----------------------
  net_out1, net_out2, z_mean, z_log_sigma_sq = encoder(input_imgs, is_train=True, reuse=False)

  # ----------------------decoder----------------------
  # decode z 
  # z = z_mean + z_sigma * eps
  z = tf.add(z_mean, tf.multiply(tf.sqrt(tf.exp(z_log_sigma_sq)), eps)) # using reparameterization tricks
  gen0, gen0_logits = generator(z, is_train=True, reuse=False) # reconstruction

  # ----------------------vgg net--------------------------
  vgg1_input = tf.image.resize_images(input_imgs,[224,224])
  net_in_real = InputLayer(vgg1_input, name='input1')
  conv1,l1_r,l2_r,l3_r,_,_ = conv_layers_simple_api(net_in_real, reuse=False)
  vgg1 = fc_layers(conv1,reuse=False)

  vgg2_input = tf.image.resize_images(gen0.outputs,[224,224])
  net_in_fake = InputLayer(vgg2_input, name='input2')
  conv2,l1,l2,l3,_,_ = conv_layers_simple_api(net_in_fake, reuse=True)
  vgg2 = fc_layers(conv2,reuse=True)


  # ----------------------for samples----------------------
  gen2, gen2_logits = generator(z, is_train=False, reuse=True)
  gen3, gen3_logits = generator(z_p, is_train=False, reuse=True)
  z_custom = tf.placeholder(tf.float32, [FLAGS['batch_size'].value, FLAGS['z_dim'].value], name='z_custom')
  gen4, gen3_logits = generator(z_custom, is_train=False, reuse=True)

  ##========================= DEFINE TRAIN OPS =======================##

  SSE_loss = tf.reduce_mean(tf.reduce_sum(tf.square(gen0.outputs - input_imgs),[1,2,3]))
  print(SSE_loss.get_shape(),type(SSE_loss))

  # perceptual loss in feature space in VGG net
  p1_loss = tf.reduce_mean(tf.reduce_sum(tf.square(l1 - l1_r), [1,2,3]))
  p2_loss = tf.reduce_mean(tf.reduce_sum(tf.square(l2 - l2_r), [1,2,3]))
  p3_loss = tf.reduce_mean(tf.reduce_sum(tf.square(l3 - l3_r), [1,2,3]))
  p_loss = p1_loss + p2_loss + p3_loss

  # train_vae
  KL_loss = tf.reduce_mean(- 0.5 * tf.reduce_sum(1 + z_log_sigma_sq - tf.square(z_mean) - tf.exp(z_log_sigma_sq),1))
  print(KL_loss.get_shape(), type(KL_loss))

  ### important points! ###
  style_content_weight = 3e-5 # you may need to tweak this weight for a different dataset
  VAE_loss = KL_loss + style_content_weight*p_loss

  e_vars = tl.layers.get_variables_with_name('encoder',True,True)
  g_vars = tl.layers.get_variables_with_name('generator', True, True)
  vae_vars = e_vars + g_vars

  print("-------encoder-------")
  net_out1.print_params(False)
  print("-------generator-------")
  gen0.print_params(False)


  # optimizers for updating encoder and generator
  vae_optim = tf.train.AdamOptimizer(lr_vae, beta1=FLAGS['beta1'].value) \
                      .minimize(VAE_loss, var_list=vae_vars)


  [TL] InputLayer  encoder/en/in: (64, 64, 64, 3)
  [TL] Conv2dLayer encoder/en/h0/conv2d: shape:[4, 4, 3, 32] strides:[1, 2, 2, 1] pad:SAME act:identity

  [TL] BatchNormLayer encoder/en/h0/batch_norm: decay:0.900000 epsilon:0.000010 act:<lambda> is_train:True
  [TL] Conv2dLayer encoder/en/h1/conv2d: shape:[4, 4, 32, 64] strides:[1, 2, 2, 1] pad:SAME act:identity
  [TL] BatchNormLayer encoder/en/h1/batch_norm: decay:0.900000 epsilon:0.000010 act:<lambda> is_train:True
  [TL] Conv2dLayer encoder/en/h2/conv2d: shape:[4, 4, 64, 128] strides:[1, 2, 2, 1] pad:SAME act:identity
  [TL] BatchNormLayer encoder/en/h2/batch_norm: decay:0.900000 epsilon:0.000010 act:<lambda> is_train:True
  [TL] Conv2dLayer encoder/en/h3/conv2d: shape:[4, 4, 128, 256] strides:[1, 2, 2, 1] pad:SAME act:identity
  [TL] BatchNormLayer encoder/en/h3/batch_norm: decay:0.900000 epsilon:0.000010 act:<lambda> is_train:True
  [TL] FlattenLayer encoder/en/h4/flatten: 4096
  [TL] DenseLayer  encoder/en/out1/lin_sigmoid: 10

In [ ]:
sess = tf.InteractiveSession()
tl.layers.initialize_global_variables(sess)

np_load_old = np.load
np.load = lambda *a,**k: np_load_old(*a, **k, allow_pickle=True)
if FLAGS['load_pretrain'].value == True:
    params_e = '/net_e_47700.npz'
    # params_e = '/net_e_28800.npz'
    # params_e = '/net_e.npz'
    load_e_params = tl.files.load_npz(path='/content/drive/My Drive/VaeOnCelebA/{}'.format('parameters_dfc_saved/dfc_vae3'), name=params_e)
    tl.files.assign_params(sess, load_e_params[:24], net_out1)
    net_out1.print_params(True)
    tl.files.assign_params(sess, np.concatenate((load_e_params[:24], load_e_params[30:]), axis=0), net_out2)
    net_out2.print_params(True)
    params_g = '/net_g_47700.npz'
    # params_g = '/net_g_28800.npz'
    # params_g = '/net_g.npz'
    load_g_params = tl.files.load_npz(path='/content/drive/My Drive/VaeOnCelebA/{}'.format('parameters_dfc_saved/dfc_vae3'),name=params_g)
    tl.files.assign_params(sess, load_g_params, gen0)
    gen0.print_params(True)

npz = np.load('/content/drive/My Drive/VaeOnCelebA/vgg16_weights.npz')
params = []
for val in sorted( npz.items() ):
    print("  Loading %s" % str(val[1].shape))
    params.append(val[1])
tl.files.assign_params(sess, params, vgg1)
tl.files.assign_params(sess, params, vgg2)

# get the list of absolute paths of all images in dataset
data_files = glob(os.path.join("./data_faces", 'img_align_celeba', "*.jpg"))
data_files = sorted(data_files)
data_files = np.array(data_files) # for tl.iterate.minibatches


  param   0: encoder/en/h0/conv2d/W_conv2d:0 (4, 4, 3, 32)      float32_ref (mean: -0.0010348871583119035, median: -0.00022342856391333044, std: 0.09539403766393661)   
  param   1: encoder/en/h0/conv2d/b_conv2d:0 (32,)              float32_ref (mean: 0.0288563035428524, median: 0.016868282109498978, std: 0.1034720242023468)   
  param   2: encoder/en/h0/batch_norm/beta:0 (32,)              float32_ref (mean: 0.013506166636943817, median: -0.011837906204164028, std: 0.23832306265830994)   
  param   3: encoder/en/h0/batch_norm/gamma:0 (32,)              float32_ref (mean: 0.9678258895874023, median: 0.970708966255188 , std: 0.15168710052967072)   
  param   4: encoder/en/h0/batch_norm/moving_mean:0 (32,)              float32_ref (mean: 0.0106828473508358, median: 0.00660718884319067, std: 0.15391750633716583)   
  param   5: encoder/en/h0/batch_norm/moving_variance:0 (32,)              float32_ref (mean: 0.15614748001098633, median: 0.11551205813884735, std: 0.16005170345306396)   
  

In [ ]:
## Produce pictures of chosen parameter set
no_picture_squares = 2
minibatch = tl.iterate.minibatches(inputs=data_files, targets=data_files, batch_size=FLAGS['batch_size'].value, shuffle=True)
for pict_no in range(no_picture_squares):
  batch_files,_ = next(minibatch)
  batch = [
    get_image(batch_file, FLAGS['image_size'].value, is_crop=FLAGS['is_crop'].value, resize_w=FLAGS['output_size'].value, is_grayscale = 0)
    for batch_file in batch_files
  ]
  batch_images = np.array(batch).astype(np.float32)

  img_recon, img_random = sess.run([gen2.outputs, gen3.outputs], feed_dict={input_imgs: batch_images})
  save_images(img_recon, [8, 8], '/content/drive/My Drive/VaeOnCelebA/parameter_samples/{}_recon.png'.format(pict_no))
  save_images(img_random, [8, 8], '/content/drive/My Drive/VaeOnCelebA/parameter_samples/{}_random.png'.format(pict_no))
  save_images(batch_images, [8, 8],'/content/drive/My Drive/VaeOnCelebA/parameter_samples/{}_original.png'.format(pict_no))
  sys.stdout.flush()


In [ ]:
SAMPLE_STEP = 10
##========================= TRAIN MODELS ================================##
iter_counter = 0

training_start_time = time.time()
# use all images in dataset in every epoch
for epoch in range(FLAGS['epoch'].value):
    minibatch = tl.iterate.minibatches(inputs=data_files, targets=data_files, batch_size=FLAGS['batch_size'].value, shuffle=True)
    idx = 0
    batch_idxs = min(len(data_files), FLAGS['train_size'].value) // FLAGS['batch_size'].value

    while True:
        try:
            batch_files,_ = next(minibatch)
            batch = [
              get_image(batch_file, FLAGS['image_size'].value, is_crop=FLAGS['is_crop'].value, resize_w=FLAGS['output_size'].value, is_grayscale = 0)
              for batch_file in batch_files
            ]
            batch_images = np.array(batch).astype(np.float32)
            vae_current_lr = FLAGS['learning_rate'].value

            # update
            p, p1, p2, p3, kl, sse, errE, _ = sess.run([p_loss,p1_loss,p2_loss,p3_loss,KL_loss,SSE_loss,VAE_loss,vae_optim], feed_dict={input_imgs: batch_images, lr_vae:vae_current_lr})

            print(
                "Epoch: [%2d/%2d] [%4d/%4d] time: %4.4f, vae_loss:%.8f, kl_loss:%.8f, sse_loss:%.8f, p1_loss:%.8f, p2_loss:%.8f, p3_loss:%.8f, p_loss:%.8f" \
                % (epoch, FLAGS['epoch'].value, idx, batch_idxs, time.time() - training_start_time, errE, kl, sse, p1, p2, p3, p))
            sys.stdout.flush()

            # save samples
            if np.mod(iter_counter, SAMPLE_STEP) == 0:
                # generate and visualize generated images
                img1, img2 = sess.run([gen2.outputs, gen3.outputs], feed_dict={input_imgs: batch_images})
                save_images(img1, [8, 8], '/content/drive/My Drive/VaeOnCelebA/{}/{:02d}_{:04d}_recon.png'.format(samples_dir, epoch, iter_counter))

                # img2 = sess.run(gen3.outputs, feed_dict={input_imgs: batch_images})
                save_images(img2, [8, 8], '/content/drive/My Drive/VaeOnCelebA/{}/{:02d}_{:04d}_random.png'.format(samples_dir, epoch, iter_counter))

                # save input image for comparison
                save_images(batch_images,[8, 8],'/content/drive/My Drive/VaeOnCelebA/{}/{:02d}_{:04d}_input_recon.png'.format(samples_dir, epoch, iter_counter))
                print("[Sample] sample generated!!!")
                sys.stdout.flush()
            # save checkpoint
            if np.mod(iter_counter, SAMPLE_STEP) == 0:
                # save current network parameters
                print("[*] Saving checkpoints...")
                net_e_name = os.path.join('/content/drive/My Drive/VaeOnCelebA/{}'.format(parameters_dir), 'net_e.npz')
                net_g_name = os.path.join('/content/drive/My Drive/VaeOnCelebA/{}'.format(parameters_dir), 'net_g.npz')
                # this version is for future re-check and visualization analysis
                net_e_iter_name = os.path.join('/content/drive/My Drive/VaeOnCelebA/{}'.format(parameters_dir), 'net_e_%d.npz' % iter_counter)
                net_g_iter_name = os.path.join('/content/drive/My Drive/VaeOnCelebA/{}'.format(parameters_dir), 'net_g_%d.npz' % iter_counter)
                # params of two branches
                net_out_params = net_out1.all_params + net_out2.all_params
                # remove repeat params
                net_out_params = tl.layers.list_remove_repeat(net_out_params)
                tl.files.save_npz(net_out_params, name=net_e_name, sess=sess)
                tl.files.save_npz(gen0.all_params, name=net_g_name, sess=sess)
                tl.files.save_npz(net_out_params, name=net_e_iter_name, sess=sess)
                tl.files.save_npz(gen0.all_params, name=net_g_iter_name, sess=sess)

                print("[*] Saving checkpoints SUCCESS!")
            iter_counter += 1
            idx += 1
            # print idx
        except StopIteration:
            print('one epoch finished')
            break

        


training_end_time = time.time()
print("The processing time of program is : {:.2f}mins".format((training_end_time-training_start_time)/60.0))

Epoch: [ 0/50] [   0/3165] time: 14.3660, vae_loss:1059.68969727, kl_loss:0.28231126, sse_loss:5543.59814453, p1_loss:622495.62500000, p2_loss:8265309.50000000, p3_loss:26425776.00000000, p_loss:35313580.00000000
[Sample] sample generated!!!
[*] Saving checkpoints...
[*] /content/drive/My Drive/VaeOnCelebA/parameters_dfc/dfc_vae3/net_e.npz saved
[*] /content/drive/My Drive/VaeOnCelebA/parameters_dfc/dfc_vae3/net_g.npz saved
[*] /content/drive/My Drive/VaeOnCelebA/parameters_dfc/dfc_vae3/net_e_0.npz saved
[*] /content/drive/My Drive/VaeOnCelebA/parameters_dfc/dfc_vae3/net_g_0.npz saved
[*] Saving checkpoints SUCCESS!
Epoch: [ 0/50] [   1/3165] time: 18.3353, vae_loss:1051.64465332, kl_loss:0.30185664, sse_loss:5269.85253906, p1_loss:594916.25000000, p2_loss:8163571.50000000, p3_loss:26286274.00000000, p_loss:35044760.00000000
Epoch: [ 0/50] [   2/3165] time: 20.8711, vae_loss:1019.64971924, kl_loss:0.36232477, sse_loss:5291.20507812, p1_loss:582199.25000000, p2_loss:7876669.00000000, p3

KeyboardInterrupt: ignored

In [ ]:
##========================= TRAIN ADAMS NN ================================##
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Input, Conv2DTranspose, Flatten, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.constraints import Constraint
import numpy as np

def loss_f(y,x):
    return tf.math.reduce_mean(tf.math.multiply(y,x))
    
def get_test_function(latent_dim, c, sigmoid = True):
    inp_layer = Input(shape=(latent_dim,))
    lay_1 = Dense(200, activation = 'tanh', kernel_constraint=Between(-c,c),bias_constraint=Between(-c,c))(inp_layer)
    lay_2 = Dense(20, activation = 'relu', kernel_constraint=Between(-c,c),bias_constraint=Between(-c,c))(lay_1)
    if sigmoid:
        out = Dense(1,activation='sigmoid',kernel_constraint=Between(-c,c),bias_constraint=Between(-c,c))(lay_2)
    else:
        out = Dense(1,kernel_constraint=Between(-c,c),bias_constraint=Between(-c,c))(lay_2)
    test_function = Model(inp_layer, out)
    test_function.compile(optimizer='adam', loss=loss_f)
    return test_function

def get_test_function1(latent_dim, c, sigmoid = True):
    inp_layer = Input(shape=(latent_dim,))
    lay_1 = Dense(2000, activation = 'relu', kernel_constraint=Between(-c,c),bias_constraint=Between(-c,c))(inp_layer)
    lay_2 = Dense(1000, activation = 'relu', kernel_constraint=Between(-c,c),bias_constraint=Between(-c,c))(lay_1)
    lay_3 = Dense(500, activation = 'relu', kernel_constraint=Between(-c,c),bias_constraint=Between(-c,c))(lay_2)
    lay_4 = Dense(250, activation = 'relu', kernel_constraint=Between(-c,c),bias_constraint=Between(-c,c))(lay_3)
    lay_5 = Dense(125, activation = 'relu', kernel_constraint=Between(-c,c),bias_constraint=Between(-c,c))(lay_4)
    lay_6 = Dense(60, activation = 'relu', kernel_constraint=Between(-c,c),bias_constraint=Between(-c,c))(lay_5)
    lay_7 = Dense(20, activation = 'relu', kernel_constraint=Between(-c,c),bias_constraint=Between(-c,c))(lay_6)
    if sigmoid:
        out = Dense(1,activation='sigmoid',kernel_constraint=Between(-c,c),bias_constraint=Between(-c,c))(lay_6)
    else:
        out = Dense(1,kernel_constraint=Between(-c,c),bias_constraint=Between(-c,c))(lay_6)
    test_function = Model(inp_layer, out)
    test_function.compile(optimizer='adam', loss=loss_f)
    return test_function

def get_test_function2(latent_dim, c, sigmoid = True):
    inp_layer = Input(shape=(latent_dim,))
    lay_1 = Dense(2500, activation = 'tanh')(inp_layer)
    lay_2 = Dense(2000, activation = 'relu')(lay_1)
    lay_3 = Dense(1500, activation = 'relu')(lay_2)
    lay_4 = Dense(1000, activation = 'relu')(lay_3)
    lay_5 = Dense(100, activation = 'relu')(lay_4)
    lay_6 = Dense(10, activation = 'relu')(lay_5)
    if sigmoid:
        out = Dense(1,activation='sigmoid')(lay_6)
    else:
        out = Dense(1)(lay_2)
    test_function = Model(inp_layer, out)
    test_function.compile(optimizer='adam', loss=loss_f)
    return test_function

def get_latent_datapoints(data_links, batch_size):
    minibatch = tl.iterate.minibatches(inputs=data_links, targets=data_links, batch_size=batch_size, shuffle=False)
    batch_files,_ = next(minibatch)
    batch = [
      get_image(
            batch_file,
            FLAGS['image_size'].value,
            is_crop=FLAGS['is_crop'].value,
            resize_w=FLAGS['output_size'].value,
            is_grayscale = 0
      )
      for batch_file in batch_files
    ]
    batch_images = np.array(batch).astype(np.float32)
    lz_samples = sess.run(z, feed_dict={input_imgs: batch_images})
    idx = 0
    while True:
      try:
        batch_files,_ = next(minibatch)
        batch = [get_image(batch_file, FLAGS['image_size'].value, is_crop=FLAGS['is_crop'].value, resize_w=FLAGS['output_size'].value, is_grayscale = 0) for batch_file in batch_files]
        batch_images = np.array(batch).astype(np.float32)
        lz_samples = np.concatenate(
              (
                  lz_samples,
                  sess.run(z, feed_dict={input_imgs: batch_images})
              )
        )
        idx +=1 
        if idx % 150 == 0:
          save_data_name = batch_files[-1][-10:]
          np.savetxt('/content/drive/My Drive/VaeOnCelebA/dfc_latent_data/' + save_data_name + ".csv", lz_samples, delimiter=",")
          print(idx)
      except StopIteration:
          print('one epoch finished')
          break
    return lz_samples

class Between(Constraint):
    def __init__(self, min_value, max_value):
        self.min_value = min_value
        self.max_value = max_value

    def __call__(self, w):
        return tf.keras.backend.clip(w, self.min_value, self.max_value)

    def get_config(self):
        return {'min_value': self.min_value,
                'max_value': self.max_value}

def train(epochs, latent_dim, test_function, batch_size, all_latent_data):
  iter = 0
  for epoch in range(epochs):
    minibatch = tl.iterate.minibatches(
        inputs=all_latent_data,
        targets=all_latent_data,
        batch_size=batch_size,
        shuffle=True
    )
    while True:
      try:
        lz_sample, _ = next(minibatch)
        z_sample = np.random.normal(size=(batch_size, latent_dim))
        y_list = np.zeros((2*batch_size,))
        y_list[0:batch_size] = -np.ones((batch_size,))
        y_list[batch_size:] = np.ones((batch_size,))
        l3 = test_function.train_on_batch(x = np.concatenate((z_sample, lz_sample)), y = y_list)
        iter += 1
        if iter % 500 == 0:
          test_function.save('/content/drive/My Drive/VaeOnCelebA/testfunction1')
          print("epoch: {}, loss: {}".format(epoch, -l3))
      except StopIteration:
          break

""" Create new testfunction """
# test_function = get_test_function(100, 1, True)
# test_function1 = get_test_function1(100, 1, True)
""" Load old testfunction """
test_function = tf.keras.models.load_model('/content/drive/My Drive/VaeOnCelebA/testfunction1', custom_objects={'Between': Between, 'loss_f': loss_f})
""" Create new latent data """
# all_latent_data = get_latent_datapoints(data_files, FLAGS['batch_size'].value)
""" Load previously created latent data """
# all_latent_data = np.loadtxt('/content/drive/My Drive/VaeOnCelebA/dfc_latent_data/201664.jpg.csv', delimiter=",")

""" Train test function on data """
# train(epochs = 10000, latent_dim=100, test_function = test_function1, batch_size = 50000, all_latent_data=all_latent_data)

' Train test function on data '

In [ ]:
def sample_f(f, a, samples):
  z_sample = sess.run(z_p, feed_dict={})
  alphas = test_function(z_p).eval()
  good_samples = z_sample[alphas.T[0] > a]
  while len(good_samples) < samples:
      z_sample = sess.run(z_p, feed_dict={})
      alphas = test_function(z_p).eval()
      good_samples = np.concatenate((good_samples, z_sample[alphas.T[0] > a]))
  return good_samples[:samples]

good_samples = sample_f(test_function, 0.95, FLAGS['batch_size'].value)
normal_samples = sess.run(z_p, feed_dict={})

good_images = sess.run(gen4.outputs, feed_dict={z_custom: good_samples})
normal_images = sess.run(gen4.outputs, feed_dict={z_custom: normal_samples})
save_images(good_images[:64], [8, 8], '/content/drive/My Drive/VaeOnCelebA/good_images.png')
save_images(normal_images[:64], [8, 8], '/content/drive/My Drive/VaeOnCelebA/normal_images.png')


In [ ]:
z_sample = sess.run(z_p, feed_dict={})
alphas = test_function(z_p).eval()
good_samples = z_sample[alphas.T[0] > 0.95]
print(np.sum(alphas.T[0] > 0.95))
print(alphas.T[0] > 0.95)
print(len(good_samples))
